In [37]:
#numpy est une bibliothèque destinée à manipuler des matrices ou tableaux multidimensionnels ainsi que des fonctions mathématiques opérant sur ces tableaux.
import numpy as np
#open3d est une bibliothèque open-source qui prend en charge le développement rapide de logiciels traitant des données 3D.
import open3d as o3d
from numpy.linalg import eig
import trimesh
#glob est utilisé pour renvoyer tous les chemins d’accès de fichier qui correspondent à un modèle spécifique.
import glob
#le module os en python fournit des fonctions de récupération du contenu des dossiers
import os 
import pandas as pd
from sklearn.decomposition import PCA 
#ce module est pour la normalisation des données
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go  
from pyntcloud import PyntCloud

In [38]:
# ###############Exporter les figures des objets originaux ###############
# for k in range(points.shape[0]):
#     obj= points[k,:,:]
#     fig = px.scatter_3d(obj,obj[:, 0], obj[:, 1], obj[:, 2]) 
#     fig.update_traces(mode='markers',marker=dict(size=1.8,color='grey',opacity=1,line=dict(width=0.01,color='DarkSlateGrey')))
#     print(k,"===============")
#     #fig.show() 
#     fig.write_html("Output/ArcadeGen/original/arcade{0}.html".format(k))

In [39]:
points = []
num_points=2048

In [56]:
#Exporter les objets 3d sous format .ply
def use_pyntcloud(pts, write_text,path):
    # ref: https://pyntcloud.readthedocs.io/en/latest/io.html
    # the doc is scarce and not complete

    n = len(pts)

    # The points must be written as a dataframe,
    # ref: https://stackoverflow.com/q/70304087/6064933
    data = {'x': pts[:, 0],
            'y': pts[:, 1],
            'z': pts[:, 2],
            'red': np.random.rand(n),
            'blue': np.random.rand(n),
            'green': np.random.rand(n)
            }

    # build a cloud
    cloud = PyntCloud(pd.DataFrame(data))

    # the argument for writing ply file can be found in
    # https://github.com/daavoo/pyntcloud/blob/7dcf5441c3b9cec5bbbfb0c71be32728d74666fe/pyntcloud/io/ply.py#L173
    filename = path + write_text+'.ply'
    cloud.to_file(filename, as_text=write_text)

    # read file
    cloud = PyntCloud.from_file(filename)
    # print(cloud)

In [40]:
def visualize_mesh(path_obj):
    mesh = o3d.io.read_triangle_mesh(path_obj)
    if mesh.is_empty(): exit()
    if not mesh.has_vertex_normals(): mesh.compute_vertex_normals()
    if not mesh.has_triangle_normals(): mesh.compute_triangle_normals()
    triangles = np.asarray(mesh.triangles)
    vertices = np.asarray(mesh.vertices)
    colors = None
    if mesh.has_triangle_normals():
        colors = (0.5, 0.5, 0.5) + np.asarray(mesh.triangle_normals) * 0.5
        colors = tuple(map(tuple, colors))
    else:
        colors = (1.0, 0.0, 0.0)
    fig = go.Figure(
        data=[
            go.Mesh3d(
                x=vertices[:,0],
                y=vertices[:,1],
                z=vertices[:,2],
                i=triangles[:,0],
                j=triangles[:,1],
                k=triangles[:,2],
                facecolor=colors,
                opacity=0.50)
        ],
        layout=dict(
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False)
            )
        )
    )
    fig.show()
def visualize_cloud(points,size=1):
    fig = go.Figure(
    data=[
        go.Scatter3d(
            x=points[:,0], y=points[:,1], z=points[:,2], 
            mode='markers',
            marker=dict(size=size, color='black')
        )
        ],
        layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
        )
        )
    fig.show()

In [41]:
files = glob.glob(os.path.join("Input/Arcade/*"))
for f in files:
    points.append(trimesh.load(f).sample(num_points))  
        
points = np.asarray(points)

unable to load materials from: ./simplify_11.obj.mtl
specified material (material_0)  not loaded!
unable to load materials from: ./simplify_12.obj.mtl
specified material (material_0)  not loaded!
unable to load materials from: ./simplify_13.obj.mtl
specified material (material_0)  not loaded!
unable to load materials from: ./simplify_14.obj.mtl
specified material (material_0)  not loaded!
unable to load materials from: ./simplify_15.obj.mtl
specified material (material_0)  not loaded!
unable to load materials from: ./simplify_16.obj.mtl
specified material (material_0)  not loaded!
unable to load materials from: ./simplify_17.obj.mtl
specified material (material_0)  not loaded!
unable to load materials from: ./simplify_18.obj.mtl
specified material (material_0)  not loaded!
unable to load materials from: ./simplify_19.obj.mtl
specified material (material_0)  not loaded!
unable to load materials from: ./simplify_20.obj.mtl
specified material (material_0)  not loaded!
unable to load mater

In [42]:
visualize_mesh('Input/Arcade/simplify_20.obj')

In [43]:
mesh = trimesh.load(os.path.join("Input/Arcade/simplify_21.obj"))
mesh.show()

unable to load materials from: ./simplify_21.obj.mtl
specified material (material_0)  not loaded!


In [44]:
M = points.shape[0]
print("Nombre des objets : ", M)

Nombre des objets :  11


In [45]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[1,:,:])
o3d.visualization.draw_geometries([pcd], window_name="Arcade Dentaire obj 11",width=500,height=500,top=50,left=700)

In [46]:
#Vous pouvez modifier le size des points dans la visualisation
visualize_cloud(points[1,:,:],1.2)

In [48]:
def pca(points,k):
    if k <= points.shape[0]:
        S = np.zeros((k, points.shape[2], points.shape[1]))
        for i in range(k):
            S[i,:,:] = np.transpose(points[i,:,:])
        #Calculant Smoy
        Smoy = np.zeros((S.shape[1],S.shape[2]))
        for i in range(S.shape[0]):
            Smoy +=  S[i,:,:]

        Smoy = Smoy / k
        Sdfs = np.zeros((k, points.shape[2],points.shape[1]))
        #Calculant les vecteurs de deformation
        for i in range(k):
            Sdfs[i,:,:] =  S[i,:,:] - Smoy
        #Matrice de covariance 
        C = np.zeros((3,3))
        for i in range(k):
            C += np.dot(Sdfs[i,:,:],np.transpose(Sdfs[i,:,:]))

        C = (1/k)*C
        w, v  = np.linalg.eigh(C)
        return w,v,Smoy
    else:
        print("We have just {0} samples".format(points.shape[0]))

In [49]:
w, v ,Smoy= pca(points,11)
print('E-value:', w)
print('E-vector', v)

E-value: [ 35847.99284009 377746.49518727 651874.81708216]
E-vector [[-0.00589271 -0.04179009 -0.99910904]
 [ 0.05672182  0.99750379 -0.04205749]
 [ 0.99837263 -0.05691911 -0.00350759]]


In [51]:
#Visualisation de la forme moyenne 
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.transpose(Smoy))
o3d.visualization.draw_geometries([pcd], window_name="Arcade Dentaire [Forme moyenne]",width=500,height=500,top=50,left=750)

In [52]:
# visualize_cloud(Smoy.T,1)

In [53]:
#Rendre la matrice des vecteurs propre une matrice orthogonale
u = np.zeros((3,3))
u[0] = v[0]
u[1] = v[1] - ((np.dot(v[1],u[0])/np.dot(u[0],u[0]))) * u[0]
u[2] = v[2] - ((np.dot(v[2],u[0])/np.dot(u[0],u[0]))) * u[0] - ((np.dot(v[2],u[1])/np.dot(u[1],u[1])) )* u[1]

In [54]:
#génerer des matrices orthogonale aléatoire
def rvs(dim=3):
     random_state = np.random
     H = np.eye(dim)
     D = np.ones((dim,))
     for n in range(1, dim):
         x = random_state.normal(size=(dim-n+1,))
         D[n-1] = np.sign(x[0])
         x[0] -= D[n-1]*np.sqrt((x*x).sum())
         # Householder transformation
         Hx = (np.eye(dim-n+1) - 2.*np.outer(x, x)/(x*x).sum())
         mat = np.eye(dim)
         mat[n-1:, n-1:] = Hx
         H = np.dot(H, mat)
         # Fix the last sign such that the determinant is 1
     D[-1] = (-1)**(1-(dim % 2))*D.prod()
     # Equivalent to np.dot(np.diag(D), H) but faster, apparently
     H = (D*H.T).T
     return H

In [57]:
augment_data1 = np.zeros((M,num_points,3))
augment_data2 = np.zeros((M,num_points,3))
for i in range(augment_data1.shape[0]):
    y = np.dot(rvs(3),np.transpose(points[i,:,:]))
    test= np.dot(v,y)
    augment_data1[i,:,:] = np.transpose(test)
    test1 = u @ np.transpose(points[i,:,:])
    augment_data2[i,:,:] = np.transpose(test1)

In [60]:
#Visualisation de la forme moyenne 
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(augment_data1[1,:,:])
o3d.visualization.draw_geometries([pcd], window_name="Arcade Dentaire [Reprojection1]",width=500,height=500,top=50,left=750)

In [63]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(augment_data2[1,:,:])
o3d.visualization.draw_geometries([pcd], window_name="Arcade Dentaire [Reprojection1]",width=500,height=500,top=50,left=750)

In [19]:
def plot3Dshape(k,size=1):
    augment_data1 = np.zeros((M,num_points,3))
    augment_data2 = np.zeros((M,num_points,3))
    for i in range(augment_data1.shape[0]):
        y = np.dot(rvs(3),np.transpose(points[i,:,:]))
        test= np.dot(v,y)
        augment_data1[i,:,:] = np.transpose(test)
        test1 = u @ np.transpose(points[i,:,:])
        augment_data2[i,:,:] = np.transpose(test1)
    now = datetime.now()
    current_time = now.strftime("%H-%M-%S") 
    now = str(current_time)
    if k < M:
        obj= points[k,:,:]
        fig = px.scatter_3d(obj,obj[:, 0], obj[:, 1], obj[:, 2]) 
        print("================Original=======================")
        #fig.show()
        visualize_cloud(obj,size)
        obj= augment_data1[k,:,:]
        fig = px.scatter_3d(obj,obj[:, 0], obj[:, 1], obj[:, 2]) 
        fig.update_traces(mode='markers',marker=dict(size=1.8,color='grey',opacity=1,line=dict(width=0.01,color='DarkSlateGrey')))
        print("================Reconstructed 1=======================")
        #fig.show()
        visualize_cloud(obj,size)
        fileno = "Gen1-"+str(k)+"--"+now
        use_pyntcloud(obj,fileno)
        fig.write_html("Output/ArcadeGen/{0}.html".format(fileno))
        obj= augment_data2[k,:,:]
        fig = px.scatter_3d(obj,obj[:, 0], obj[:, 1], obj[:, 2]) 
        print("================Reconstructed 2=======================")
        #fig.show()
        visualize_cloud(obj,size)
        fileno = "Gen2-"+str(k)+"--"+now
        use_pyntcloud(obj,fileno)
        fig.update_traces(mode='markers',marker=dict(size=1.8,color='grey',opacity=1,line=dict(width=0.01,color='DarkSlateGrey')))
        fig.write_html("Output/ArcadeGen/{0}.html".format(fileno))
        
    else:
        print("We have just {0} samples".format(M))

In [20]:
plot3Dshape(10,1.5)

================Original=======================


================Reconstructed 1=======================


================Reconstructed 2=======================


In [34]:
def plot_3d(k,points):
    augment_data1 = np.zeros((M,num_points,3))
    augment_data2 = np.zeros((M,num_points,3))
    for i in range(augment_data1.shape[0]):
        y = rvs(3) @ np.transpose(points[i,:,:])
        test= np.dot(v,y)
        augment_data1[i,:,:] = np.transpose(test)
        test1 = u @ np.transpose(points[i,:,:])
        augment_data2[i,:,:] = np.transpose(test1)
    obj = points[k,:,:]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(obj)
    print("======================================= 1 =======================================")
    o3d.visualization.draw_geometries([pcd],window_name="Original Dental Arch",width=500,height=500,top=50,left=700)
    obj = augment_data1[k,:,:]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(obj)
    print("======================================= 2 =======================================")
    o3d.visualization.draw_geometries([pcd],window_name="Reconstructed Dental Arch 1",width=500,height=500,top=50,left=700)
    obj = augment_data2[k,:,:]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(obj)
    print("======================================= 3 =======================================")
    o3d.visualization.draw_geometries([pcd],window_name="Reconstructed Dental Arch 2",width=500,height=500,top=50,left=700)
 

In [35]:
plot_3d(10,points)

======================================= 1 =======================================
======================================= 2 =======================================
======================================= 3 =======================================
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [43]:
#verification que les objets sont pas les memes
augment_data1 = np.zeros((M,num_points,3))
augment_data2 = np.zeros((M,num_points,3))
for i in range(augment_data1.shape[0]):
    y = rvs(3) @ np.transpose(points[i,:,:])
    test= np.dot(v,y)
    augment_data1[i,:,:] = np.transpose(test)
    test1 = u @ np.transpose(points[i,:,:])
    augment_data2[i,:,:] = np.transpose(test1)

In [44]:
def verif(c=0,k=0):
    c = 0
    t1 = augment_data1[k,:,:]
    t2 = points[k,:,:]
    t3 = augment_data2[k,:,:]
    for i in range(num_points):
        if t1[i,:] in t2 or t3[i,:] in t2:
            c = c + 1
    print("obj {0} :{1}".format(k,c))

In [45]:
for i in range(M):
    verif(0,i)

obj 0 :0
obj 1 :0
obj 2 :0
obj 3 :0
obj 4 :0
obj 5 :0
obj 6 :0
obj 7 :0
obj 8 :0
obj 9 :0
obj 10 :0


In [191]:
#export original data
# for i in range(points.shape[0]):
#     obj= points[i,:,:]
#     fig = px.scatter_3d(obj,obj[:, 0], obj[:, 1], obj[:, 2]) 
#     print("=================== Original Object {0} =======================".format(i+1))
#     fig.show()
#     fileno = "original"+str(i) 
#     fig.write_html("Output/ArcadeGen/original/{0}.html".format(fileno))